# 如何用最少的次数测出鸡蛋会在哪一层摔碎？


https://www.zhihu.com/question/19690210/answer/18079633

一幢 200 层的大楼,给你两个鸡蛋. 如果在第 n 层扔下鸡蛋,鸡蛋不碎,那么从前 n-1 层扔鸡蛋都不碎. 这两只鸡蛋一模一样,不碎的话可以扔无数次. 已知鸡蛋在0层扔不会碎.

提出一个策略, 要**保证**能测出鸡蛋**恰好会碎**的楼层, 并使此策略在**最坏情况下所扔次数最少**.

题目严谨描述:
一幢 200 层的大楼,给你两个鸡蛋. 如果在第 n 层扔下鸡蛋,鸡蛋不碎,那么从前 n-1 层扔鸡蛋都不碎.
这两只鸡蛋一模一样,不碎的话可以扔无数次. 已知鸡蛋在0层扔不会碎.
提出一个策略, 要**保证**能测出鸡蛋**恰好会碎**的楼层, 并使此策略在**最坏情况下所扔次数最少**.

搞清楚这题的意思:
第一个鸡蛋用来试探, 只要它从 k 层楼扔下去没碎, 则目标就在[k+1, 200]之间了.
但一旦运气不好碎了, 对于已知的区间, 我们只能用剩下一个鸡蛋从小到大一层层试,
因为我们要保证策略必须成功, 不能冒险了.

"最坏情况下代价最小"这句话十分重要, 它反映了题目的重要数学结构:
我们可以把任何一种策略都看成一个决策树,
每一次扔瓶子都会有两个子节点, 对应碎与不碎的情况下下一步应该扔的楼层.
那么, 策略的一次执行, 是树中的一条从根往下走的路,
当且仅当这条路上出现过形如 k 没碎 与 k+1 碎了的一对节点时, 路停止, 当前节点不再扩展.
那么要找的是这么一棵树, 使得所有路里最长者尽量短, 也即, 要找一个最矮的决策树.

再看一个节点处, 选择楼层时会发生什么.
容易看出, 选择的楼层如果变高, 那么"碎子树"高度不减, "不碎子树"高度不增.
同样的, 选择的楼层变矮的话, "碎子树"高度不增, "不碎子树"高度不减.

这时候答案很明显了: 为了使两子树中高度最大者尽量小, 我们的选择应当使两子树高度尽量接近.
最终希望的结果是, 整个二叉树尽量像一个满二叉树.

假设第一次在根节点上, 我们选择扔$k$层, 其"碎子树"的高度显然是$k - 1$.为了考虑不碎子树的高度, 设不碎后第二次扔$m$层(显然$m > k$ ),

则这个新节点的碎子树高度为 $m - k - 1$, 不碎子树高度仍然未知,但按照满二叉树的目标, 我们认为它与碎子树相同或少1就好.

那么在根节点上的不碎子树的高度就是$m -k-1 + 1$, 令它与碎子树高度相同, 于是:

$m - k - 1 + 1 = k - 1$ => $m = k + k - 1$

也即, 如果第一次扔在k层, 第二次应该高k-1 层, 这可以有直观一点的理解:
每扔一次, 就要更保守一些, 所以让区间长度少1. $[1, k) -> [k + 1, 2k - 1)$

用类似刚才的分析, 可以继续得到, 下一次应该增高k - 2, 再下一次应该增高k - 3.

如果大楼100层, 考虑:
![[公式]](https://www.zhihu.com/equation?tex=k+%2B+%28k-1%29+%2B+%5Ccdots+%2B+1+%3D+%5Cfrac%7Bk%28k%2B1%29%7D%7B2%7D+%3D+100+%5CRightarrow+k+%5Capprox+14)

所以第一次扔14层, 最坏需要14次(策略不唯一, 树的叶子可以交换位置).200层的话, 类似得到k =20.

以上是数学做法...当然还有代码做法....
设f(n, m)为n层楼, m个蛋所需次数, 那么它成了一道DP题..

![[公式]](https://www.zhihu.com/equation?tex=%5Cbegin%7Beqnarray%7D%0Af%280%2C+m%29+%26+%3D+%26+0%2C+%28m+%3E%3D+1%29%5C%5C%0Af%28n%2C+1%29+%26+%3D+%26+n%2C+%28n+%3E%3D+1%29%5C%5C%0Af%28n%2C+m%29+%26+%3D+%26+%5Cmin_%7B1+%5Cle+i+%5Cle+n%7D+%5C%7B+%5Cmax%5C%7B+f%28i+-+1%2C+m+-+1%29%2C+f%28n+-+i%2C+m%29%5C%7D%5C%7D+%2B+1+%5C%5C%0A%5Cend%7Beqnarray%7D)


n层楼, m个蛋扩展动态规划的解释：

https://zhuanlan.zhihu.com/p/41257286

**python3的functools的一个自带函数, 可以对函数返回结果进行LRU cache, 下次以相同参数调用就不重复计算了**

**maxsize=None 不限制大小, 其实就变成是全部都cache下来, 不考虑LRU了**

In [2]:
import functools
@functools.lru_cache(maxsize=None)
def f(n, m): #n层楼，m个鸡蛋
    if n == 0:
        return 0
    if m == 1:
        return n

    ans = min([max([f(i - 1, m - 1), f(n - i, m)]) for i in range(1, n + 1)]) + 1
    # 先拿鸡蛋在第i层试一次（最后的+1的由来）
    # 鸡蛋碎，则剩余i-1楼层、m-1鸡蛋；鸡蛋不碎，则剩余n-i楼层，m个鸡蛋。找到两者中最大值
    # 从1到n逐个试验，找到各个最大值的最小值。
    return ans

In [4]:
print(f(100, 2))
print(f(200, 2))
print(f(200,3))

14
20
11


## 优秀回答，扩展

如何用最少的次数测出鸡蛋会在哪一层摔碎？ - 刘遥行的回答 - 知乎

https://www.zhihu.com/question/19690210/answer/514119129